# The ETL Process - Extract (E)

We will be performing hands-on data extraction using the following Python libraries for web scraping: 

- **Requests:** For sending HTTP requests.

- **Beautiful Soup:** For parsing HTML content.

Let us start by installing the above mentioned libraries using Python's package manager (pip).

In [1]:
# !pip install requests
# !pip install beautifulsoup4 

Once the installation has been finished, we can import the libraries in Python using the following import convention.

In [2]:
# Library import convention
import requests
from bs4 import BeautifulSoup

Now, let us recall the web scraping workflow.

- **Step 1:** Request the target website for its content.

<div ><img src="images/scraping_1.PNG" width="500"></div>

- **Step 2:** Receive the HTML content from the target website.

<img src="images/scraping_2.PNG" width="500">

- **Step 3:** Parse the HTML content and extract the necessary data.

<img src="images/scraping_3.PNG" width="500">

**The Target Website: Estes Park Weather website**

We are using the weather reports data from Estes Park Weather website, where the data is available in a monthly format for each day of the month. For the purpose of this tutorial we are taking the data for the month of May 2020 (in the format 202005).

"https://www.estesparkweather.net/archive_reports.php?date=202005"

<img src="images/weather_website.PNG" width="500">

The first step is to send a request to the target website for receiving its content as a response. This can be done using the ```get``` function of the requests library.

In [3]:
# URL of the target website
url = "https://www.estesparkweather.net/archive_reports.php?date=202005"

# Request the target website for its content and get a response back
response = requests.get(url)

# A 200 response status means a successful response 
print(response)

<Response [200]>


In [4]:
# Viewing the first 500 character content of the page
print(response.content[:500])

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<!-- DW6 -->\n<head>\n<!-- Copyright 2005 Macromedia, Inc. All rights reserved. -->\n<meta http-equiv="Refresh" content="300" />\n<meta http-equiv="Pragma" content="no-cache" />\n<meta http-equiv="Cache-Control" content="no-cache" />\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />\n<meta name="ICBM" content="37.27465, -122.'


After getting the response, we will now parse the received content using the HTML parser from Beautiful Soup.

In [5]:
# Get page content in the form of HTML using Beautiful Soup's HTML parser
soup =  BeautifulSoup(response.content,'html.parser')

We can extract data values from the table for the month of May 2020 using the `find_all()` method of Beautiful Soup. 

In [6]:
# Find all data under the tag 'table'
table_data = soup.find_all('table')

In [7]:
# Print the first table data
print(table_data[0])

<table border="0" cellpadding="3" cellspacing="0">
<tr class="table-top">
<td colspan="2">May 1 Average and Extremes</td>
</tr>
<tr class="column-light">
<td>Average temperature</td><td> 57.8°F</td>
</tr>
<tr class="column-dark">
<td>Average humidity</td><td> 45%</td>
</tr>
<tr class="column-light">
<td>Average dewpoint</td><td> 33.9°F</td>
</tr>
<tr class="column-dark">
<td>Average barometer</td><td> 29.6 in.</td>
</tr>
<tr class="column-light">
<td>Average windspeed</td><td> 5.2 mph</td>
</tr>
<tr class="column-dark">
<td>Average gustspeed</td><td> 8.0 mph</td>
</tr>
<tr class="column-light">
<td>Average direction</td><td> 238° (WSW)</td>
</tr>
<tr class="column-dark">
<td>Rainfall for month</td><td> 0.00 in.</td>
</tr>
<tr class="column-light">
<td>Rainfall for year</td><td> 2.40 in.</td>
</tr>
<tr class="column-dark">
<td>Maximum rain per minute</td><td> 0.00 in. on day 01 at time 00:00</td>
</tr>
<tr class="column-light">
<td>Maximum temperature</td><td> 71.8°F on day 01 at time 1

In [8]:
# Get the text from the table data
table_data[0].text

'\n\nMay 1 Average and Extremes\n\n\nAverage temperature 57.8°F\n\n\nAverage humidity 45%\n\n\nAverage dewpoint 33.9°F\n\n\nAverage barometer 29.6 in.\n\n\nAverage windspeed 5.2 mph\n\n\nAverage gustspeed 8.0 mph\n\n\nAverage direction 238° (WSW)\n\n\nRainfall for month 0.00 in.\n\n\nRainfall for year 2.40 in.\n\n\nMaximum rain per minute 0.00 in. on day 01 at time 00:00\n\n\nMaximum temperature 71.8°F on day 01 at time 14:41\n\n\nMinimum temperature 41.5°F on day 01 at time 06:24\n\n\nMaximum humidity 83% on day 01 at time 00:00\n\n\nMinimum humidity 19% on day 01 at time 14:07\n\n\nMaximum pressure 29.724 in. on day 01 at time 22:22\n\n\nMinimum pressure 29.515 in. on day 01 at time 16:13\n\n\nMaximum windspeed 18.4 mph on day 01 at time 21:02\n\n\nMaximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57\n\n\nMaximum heat index 76.6°F on day 01 at time 15:57\n\n'

The text data is essentially a large string which is hard to read.

# The ETL Process - Transform (T)

Next, we will learn how to transform the data to make it proper enough for loading into the data warehouse (in our case, a database).

We will be performing hands-on transformation using the following Python libraries: 

- **Re:** For string matching using Regular expressions.

- **Datetime:** For date manipulation.

We can import the libraries in Python using the following import convention.

In [9]:
# Library import convention
import re
from datetime import datetime

We left off the last lesson at the point where we viewed the data from one of the tables we extracted from the parsed HTML.

In [10]:
# Get the text from the table data
table_data[0].text

'\n\nMay 1 Average and Extremes\n\n\nAverage temperature 57.8°F\n\n\nAverage humidity 45%\n\n\nAverage dewpoint 33.9°F\n\n\nAverage barometer 29.6 in.\n\n\nAverage windspeed 5.2 mph\n\n\nAverage gustspeed 8.0 mph\n\n\nAverage direction 238° (WSW)\n\n\nRainfall for month 0.00 in.\n\n\nRainfall for year 2.40 in.\n\n\nMaximum rain per minute 0.00 in. on day 01 at time 00:00\n\n\nMaximum temperature 71.8°F on day 01 at time 14:41\n\n\nMinimum temperature 41.5°F on day 01 at time 06:24\n\n\nMaximum humidity 83% on day 01 at time 00:00\n\n\nMinimum humidity 19% on day 01 at time 14:07\n\n\nMaximum pressure 29.724 in. on day 01 at time 22:22\n\n\nMinimum pressure 29.515 in. on day 01 at time 16:13\n\n\nMaximum windspeed 18.4 mph on day 01 at time 21:02\n\n\nMaximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57\n\n\nMaximum heat index 76.6°F on day 01 at time 15:57\n\n'

Without cleaning the data, it would be really difficult to load it into the database. Let us transform the data and change it into a suitable format.

The text contains a lot of new line strings denoted by \n. So we will be filtering them to get only the required data.

In [11]:
# Filter and remove the empty strings in the list
list(filter(None, table_data[0].text.splitlines()))

['May 1 Average and Extremes',
 'Average temperature 57.8°F',
 'Average humidity 45%',
 'Average dewpoint 33.9°F',
 'Average barometer 29.6 in.',
 'Average windspeed 5.2 mph',
 'Average gustspeed 8.0 mph',
 'Average direction 238° (WSW)',
 'Rainfall for month 0.00 in.',
 'Rainfall for year 2.40 in.',
 'Maximum rain per minute 0.00 in. on day 01 at time 00:00',
 'Maximum temperature 71.8°F on day 01 at time 14:41',
 'Minimum temperature 41.5°F on day 01 at time 06:24',
 'Maximum humidity 83% on day 01 at time 00:00',
 'Minimum humidity 19% on day 01 at time 14:07',
 'Maximum pressure 29.724 in. on day 01 at time 22:22',
 'Minimum pressure 29.515 in. on day 01 at time 16:13',
 'Maximum windspeed 18.4 mph on day 01 at time 21:02',
 'Maximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57',
 'Maximum heat index 76.6°F on day 01 at time 15:57']

This is the data for May 1.

Since we want to extract the data for the entire month of May, let us do the same for every day of the month and save it as a nested list.

[[Data for May 1], [Data for May 2], ... [Data for May 31]]

In [12]:
# Split the data and save as a list
outer_data_list = []

# Looping through each day's data
for data in table_data[:31]:
    
    # Get filtered value without empty strings
    value = list(filter(None, data.text.splitlines()))
    
    # Append each day values to the list
    outer_data_list.append(value)

In [13]:
outer_data_list[0]

['May 1 Average and Extremes',
 'Average temperature 57.8°F',
 'Average humidity 45%',
 'Average dewpoint 33.9°F',
 'Average barometer 29.6 in.',
 'Average windspeed 5.2 mph',
 'Average gustspeed 8.0 mph',
 'Average direction 238° (WSW)',
 'Rainfall for month 0.00 in.',
 'Rainfall for year 2.40 in.',
 'Maximum rain per minute 0.00 in. on day 01 at time 00:00',
 'Maximum temperature 71.8°F on day 01 at time 14:41',
 'Minimum temperature 41.5°F on day 01 at time 06:24',
 'Maximum humidity 83% on day 01 at time 00:00',
 'Minimum humidity 19% on day 01 at time 14:07',
 'Maximum pressure 29.724 in. on day 01 at time 22:22',
 'Minimum pressure 29.515 in. on day 01 at time 16:13',
 'Maximum windspeed 18.4 mph on day 01 at time 21:02',
 'Maximum gust speed 29.9 mph  from 063 °(ENE) on day 01 at time 20:57',
 'Maximum heat index 76.6°F on day 01 at time 15:57']

To get the numerical value of the data from the list of strings, we will have to perform string matching using regular expression.

As an example, let us see how we can match a string using a regular expression by taking only a single string element from the list of strings. In this case, we are taking the element at index 10.

In [14]:
outer_data_list[0][10]

'Maximum rain per minute 0.00 in. on day 01 at time 00:00'

Since we are only concerned about the data for the 'Maximum rain per minute', let us only select the first five words.

In [15]:
outer_data_list[0][10].split()[:5]

['Maximum', 'rain', 'per', 'minute', '0.00']

We can now use the findall() function of the regular expression library to find decimal values in the string.

In [16]:
# Use regex to find the digits from the string 
re.findall("\d+",str(outer_data_list[0][10].split()[:5]))

['0', '00']

Let us convert the decimal values as it was before.

In [17]:
# Join the strings with a decimal
'.'.join(re.findall("\d+",str(outer_data_list[0][10].split()[:5])))

'0.00'

Finally, we will now get the data values for each day of the month May like we did above and save them into different list formats along with the corresponding dates.

[[Numeric Values List for May 1], 

[Numeric Values List for May 2], ..., 

[Numeric Values List for May 31] ]

In [18]:
# List to hold data values for each day
day_data_list = []

# Looping through each day
for i in outer_data_list:
    
    # List to hold numeric values
    numeric_values = []

    # Looping through all the data elements of each day
    for day_data in i[1:]:
        
        # Parse the text to get only digits
        d = '.'.join(re.findall("\d+",str(day_data.split()[:5])))
        
        # Appending to the numeric values list
        numeric_values.append(d)
    
    # Appending to the data values for each day
    day_data_list.append(numeric_values)

Now, we will be storing the data into a dataframe for easy loading into a database.

Let us install the [Pandas](https://pandas.pydata.org/) library for data manipulation. Pandas is a fast, powerful, flexible, high-level data analysis and manipulation tool, that is built on top of the Python programming language. You can learn more about it [here](https://www.theclickreader.com/courses/learn-pandas-for-data-science/).

In [20]:
# !pip install pandas

We can then import it as pd.

In [21]:
import pandas as pd

A **DataFrame** is a 2-dimensional labeled data structure in Pandas which is similar to a table. It contains rows and columns of different types of data. 

We will first get all the column names from the website and save it as a list named columns for our dataframe.

<img src="images/weather_website.PNG" width="500">

In [22]:
# Name of the columns as per the website
columns = ['Average temperature', 'Average humidity', 'Average dewpoint', 'Average barometer', 'Average windspeed', 'Average gustspeed', 'Average direction', 'Rainfall for month', 'Rainfall for year', 'Maximum rain per minute', 'Maximum temperature', 'Minimum temperature', 'Maximum humidity', 
           'Minimum humidity', 'Maximum pressure', 'Minimum pressure', 'Maximum windspeed', 'Maximum gust speed', 'Maximum heat index']

Let us now create the dataframe by adding our ```data_list``` as the data and ```columns``` as the columns of the dataframe.

In [23]:
# Create a pandas dataframe with the data
weather_data = pd.DataFrame(day_data_list, columns = columns)

In [24]:
weather_data.head()

,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index
0,57.8,45,33.9,29.6,5.2,8.0,238,0.00,2.40,0.00,71.8,41.5,83,19,29.724,29.515,18.4,29.9,76.6
1,47.1,83,42.0,29.7,3.1,5.0,16,0.30,2.70,0.02,57.3,39.9,95,60,29.881,29.666,15.0,19.6,57.3
2,51.6,59,36.4,29.7,4.9,7.5,244,0.33,2.73,0.01,65.4,36.1,93,29,29.876,29.620,19.6,26.5,65.4
3,45.1,58,28.9,29.8,7.6,11.8,248,0.37,2.77,0.01,52.2,38.5,94,28,30.024,29.604,25.3,36.8,52.2
4,48.7,24,12.2,30.0,6.3,9.7,241,0.37,2.77,0.00,62.3,37.9,50,12,30.128,29.942,18.4,26.5,62.3


Similarly, the date list that we created can be simply added as a new column named "Date" to the dataframe as follows.

In [25]:
# Add a new column with the date values
weather_data['Date'] = pd.date_range('2020-05-01', '2020-05-31')

In [26]:
# View the data
weather_data.head()

,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index,Date
0,57.8,45,33.9,29.6,5.2,8.0,238,0.00,2.40,0.00,71.8,41.5,83,19,29.724,29.515,18.4,29.9,76.6,2020-05-01
1,47.1,83,42.0,29.7,3.1,5.0,16,0.30,2.70,0.02,57.3,39.9,95,60,29.881,29.666,15.0,19.6,57.3,2020-05-02
2,51.6,59,36.4,29.7,4.9,7.5,244,0.33,2.73,0.01,65.4,36.1,93,29,29.876,29.620,19.6,26.5,65.4,2020-05-03
3,45.1,58,28.9,29.8,7.6,11.8,248,0.37,2.77,0.01,52.2,38.5,94,28,30.024,29.604,25.3,36.8,52.2,2020-05-04
4,48.7,24,12.2,30.0,6.3,9.7,241,0.37,2.77,0.00,62.3,37.9,50,12,30.128,29.942,18.4,26.5,62.3,2020-05-05


In [27]:
# Convert data into numeric
weather_data[columns] = weather_data[columns].apply(pd.to_numeric)

We will now format the name of the columns by replacing the space by underscore and converting it into lowercase.

In [28]:
weather_data.rename(columns = lambda x: x.replace(" ", "_").lower(),
          inplace = True)

# Check the column names of the dataframe
weather_data.keys()

Index(['average_temperature', 'average_humidity', 'average_dewpoint',
       'average_barometer', 'average_windspeed', 'average_gustspeed',
       'average_direction', 'rainfall_for_month', 'rainfall_for_year',
       'maximum_rain_per_minute', 'maximum_temperature', 'minimum_temperature',
       'maximum_humidity', 'minimum_humidity', 'maximum_pressure',
       'minimum_pressure', 'maximum_windspeed', 'maximum_gust_speed',
       'maximum_heat_index', 'date'],
      dtype='object')

Humidity is in percentage as mentioned in the website. We will hence divide each data value by 100.

In [29]:
humidity_cols = ['average_humidity', 'maximum_humidity', 'minimum_humidity']

# Dividing the value by 100 to get the percentage
weather_data[humidity_cols] = weather_data[humidity_cols].apply(lambda x: x/100)

In [30]:
# data after division
weather_data.head()

,average_temperature,average_humidity,average_dewpoint,average_barometer,average_windspeed,average_gustspeed,average_direction,rainfall_for_month,rainfall_for_year,maximum_rain_per_minute,maximum_temperature,minimum_temperature,maximum_humidity,minimum_humidity,maximum_pressure,minimum_pressure,maximum_windspeed,maximum_gust_speed,maximum_heat_index,date
0,57.8,0.45,33.9,29.6,5.2,8.0,238,0.00,2.40,0.00,71.8,41.5,0.83,0.19,29.724,29.515,18.4,29.9,76.6,2020-05-01
1,47.1,0.83,42.0,29.7,3.1,5.0,16,0.30,2.70,0.02,57.3,39.9,0.95,0.60,29.881,29.666,15.0,19.6,57.3,2020-05-02
2,51.6,0.59,36.4,29.7,4.9,7.5,244,0.33,2.73,0.01,65.4,36.1,0.93,0.29,29.876,29.620,19.6,26.5,65.4,2020-05-03
3,45.1,0.58,28.9,29.8,7.6,11.8,248,0.37,2.77,0.01,52.2,38.5,0.94,0.28,30.024,29.604,25.3,36.8,52.2,2020-05-04
4,48.7,0.24,12.2,30.0,6.3,9.7,241,0.37,2.77,0.00,62.3,37.9,0.50,0.12,30.128,29.942,18.4,26.5,62.3,2020-05-05


#### Saving data as a csv

The dataframe can be saved as csv using the [`to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) function of the Pandas dataframe.

In [31]:
weather_data.to_csv("data/weather_data.csv", index=False)

# The ETL Process - Load (L)

The final step in the ETL (Extract-Transform-Load) process is to load the data into a data warehouse (database). We will learn how to do so by using the SQLite library in Python.

We can install the SQLite library using the Python package manager by using the following command.

In [33]:
# !pip install db-sqlite3

To import sqlite in Python, just write the following code and run it.

In [34]:
import sqlite3

In order to create the database, we will first need to open a connection using the connect() function of the [sqlite](https://docs.python.org/2/library/sqlite3.html) module.

**Note:** Make sure a directory named "data" exists in your working directory.

In [36]:
# connect function opens a connection to the SQLite database file 
conn = sqlite3.connect('data/weather_data.db')
print("Database connection opened.")

Database connection opened.


## Creating a table 

In order to create a table, we will be using the execute() function with the SQL command `CREATE` as follows.

In [37]:
# Create a new Table named as Weather with the columns
try:
    conn.execute('''
        CREATE TABLE Weather
        (
        average_temperature         Float DEFAULT 0,
        average_humidity            Float DEFAULT 0,
        average_dewpoint            Float DEFAULT 0,
        average_barometer           Float DEFAULT 0,
        average_windspeed           Float DEFAULT 0,
        average_gustspeed           Float DEFAULT 0,
        average_direction           Float DEFAULT 0,
        rainfall_for_month          Float DEFAULT 0, 
        rainfall_for_year           Float DEFAULT 0, 
        maximum_rain_per_minute     Float DEFAULT 0, 
        maximum_temperature         Float DEFAULT 0,
        minimum_temperature         Float DEFAULT 0, 
        maximum_humidity            Float DEFAULT 0, 
        minimum_humidity            Float DEFAULT 0, 
        maximum_pressure            Float DEFAULT 0, 
        minimum_pressure            Float DEFAULT 0,
        maximum_windspeed           Float DEFAULT 0, 
        maximum_gust_speed          Float DEFAULT 0, 
        maximum_heat_index          Float DEFAULT 0, 
        date                        timestamp);''')
    
    print ("Weather table has been created successfully");
    
except Exception as e:
    # Print the exception error message
    print(str(e))
    print('Table Creation Failed.')
    
finally:
    # Finally close the database connection
    conn.close() 

Weather table has been created successfully


We will now get the values of the dataframe as a list to insert them into the database.

In [38]:
# Convert pandas dataframe to list of list
weather_data_list = weather_data.astype('str').values.tolist()

Let us first initiate a new connection to the database.

In [39]:
# Make new connection to Insert data in SQL DB
conn = sqlite3.connect('data/weather_data.db')

We will then create a cursor from the connection to use it for querying the database.

In [40]:
# Make a cursor for querying SQL database
cur = conn.cursor()

## Insert values from list to the database

We will now insert the list into the database using the SQL command `INSERT`.

In [41]:
try:
    # will use ? sign to represent each column names inside VALUE().
    cur.executemany("INSERT INTO Weather(average_temperature, average_humidity, average_dewpoint,\
                    average_barometer, average_windspeed, average_gustspeed, average_direction,\
                    rainfall_for_month, rainfall_for_year, maximum_rain_per_minute, maximum_temperature,\
                    minimum_temperature, maximum_humidity, minimum_humidity, maximum_pressure, minimum_pressure,\
                    maximum_windspeed, maximum_gust_speed, maximum_heat_index, date) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", weather_data_list)
    conn.commit()
    print('Weather Data Inserted Successfully')
    
except Exception as e:
    # Print the exception error message
    print(str(e))
    print('Weather Data Insertion Failed')
    
finally:
    # finally close the database connection
    conn.close()

Weather Data Inserted Successfully


We can see that the data has been inserted successfully which can be viewed through the [DB Browser for Sqlite](https://sqlitebrowser.org/dl/).

<img src="images/weather_database.PNG">

We have successfully loaded our extracted data onto a database. 

#### With this, we conclude our lesson on the ETL process.